In [ ]:
#!pip uninstall pyspark
H={'/home/user/HADOOP': 'C:\HADOOP',
  '/home/user/HADOOP/spark-2.3.0-bin-hadoop2.7': 'C:\HADOOP\spark-2.3.0-bin-hadoop2.7',
  '/home/user/Program Files/java/8.0.172-zulu': 'C:\Program Files\Java\jdk1.8.0_111',
  '/home/user/TMP/SPARK': 'C:\TMP\SPARK'}
#H=dict([(k,k) for k in H.keys()])
Z="ok"
SB="SANDBOX.USER"
DB="MY_DATA"
TB="CURRENCY_KAFKA"
BA="SANDBOX.USER"
AB="REQUEST_CURRENCY"
CB="SANDBOX.USER"
BC="RESPONSE_CURRENCY"
DIR="C:/ALBINA/SASHA/SOAP"
C=[("dt","","")
  ,("val","FLOAT","cast(val as float)")
]
AKEY="mqZagQhmMKp1w1-565-E"
URL='https://www.quandl.com/api/v3/datasets/BOE/XUDLBK69/data.json'

In [2]:
#%%time

def parse_html(url, nrows=-1):
    import pandas as pd
    d = pd.DataFrame(columns=['dt', 'val'])
    import requests
    answer = requests.get(url+'?api_key='+AKEY, stream=True)
    res = answer.text
    import json
    js = json.loads(res)
    data = js["dataset_data"]["data"]
    for i in range(len(data)):
        if len(d) >= nrows and nrows >= 0: break
        row = []
        row.append(data[i][0])
        row.append(data[i][1])
        d.loc[len(d)+1] = row
    return d

print(Z)

ok


In [3]:
import pandas as pd
view = parse_html(URL, nrows=100)
display(view[view.index < 5])

print(Z)

,dt,val
1,2018-07-10,62.1450
2,2018-07-09,62.7100
3,2018-07-06,62.9750
4,2018-07-05,63.1075


ok


In [4]:
%%time

import os
os.environ['HADOOP_HOME'] = H['/home/user/HADOOP']
os.environ['SPARK_HOME'] = H['/home/user/HADOOP/spark-2.3.0-bin-hadoop2.7']
os.environ['JAVA_HOME'] = H['/home/user/Program Files/java/8.0.172-zulu']

def start():
    import time
    from pyspark.sql import SparkSession
    res = SparkSession.builder.master('local[*]').appName('test') \
       .config('spark.sql.warehouse.dir', 'file:///' + WARE) \
       .config("spark.driver.maxResultSize", "600g") \
       .config("spark.sql.shuffle.output.partitions", "32768") \
       .getOrCreate()
    time.sleep(1)
    return res

def stop(sc):
    print("ok")
    sc.stop()

def preview(df):
    df.show(n=3, truncate=False)

WARE=H['/home/user/TMP/SPARK']
os.makedirs(WARE, exist_ok=True)
os.chmod(WARE, 0o777)
spark=start()
print(spark.version)

print(Z)

ModuleNotFoundError: No module named 'pyspark'

In [5]:
%%time

def spark_table(name, sqlc):
    import pyspark.storagelevel as L
    import shutil
    import pandas as pd
    if name == SB+'_'+TB:
        shutil.copyfile(DIR+"/"+"manyphones.tsv", WARE+"/igDOL_"+name+".csv")
    else:
        shutil.rmtree(WARE+"/igDOL_"+name+".csv")
    q = ["cast(_c"+str(i)+" as string) as " + C[i][0] for i in range(len(C))]
    qq = [(C[i][2] if len(C[i][2]) else C[i][0])+' as '+C[i][0] for i in range(len(C))]
    return sqlc.read.option("escape", "\"").option("quote", "\"").option("sep", "\t").csv(WARE+"/igDOL_"+name+".csv",
                header=False).persist(L.StorageLevel.MEMORY_AND_DISK).repartition(1).selectExpr(q).selectExpr(qq)

def write_insert(df, dst):
    df.write.option("escape", "\"").option("quote", "\"").csv(WARE+"/igDOL_"+dst, header=False, mode="Overwrite")

print(Z)

ok
Wall time: 0 ns


In [6]:
#%%time

import pyspark.sql.functions as F
import pyspark.sql.window as W
import pyspark.sql.types as T

df = spark_table(SB+"_"+TB, spark)
write_insert(df.coalesce(1), SB+"."+TB)
preview(df)

print(Z)

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
%%time
a=[]
a.append(("load_tags", "", ""))
a.append(("load_rank", "BIGINT", "cast(load_rank as bigint)"))
print("(1) rename "+WARE+"/igDOL_"+SB+"."+TB)
print("part.csv to")
print("many.csv")
print("")
print("")
print("(2) upload to ambari (permissions 444)")
print("make folder")
print("/user/yurbasov/tmp/phones_many")
print("permissions 777")
print("")
print("")
print("(3) CREATE EXTERNAL TABLE "+SB+"_"+TB+"(")
print('\n,'.join(['`'+C[i][0]+'`  string' for i in range(len(C))]))
print(")")
print("ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'")
print("WITH SERDEPROPERTIES (")
print("\"separatorChar\" = \",\",")
print("\"quoteChar\"     = \"\\\"\"")
print(") STORED as textfile LOCATION '/user/yurbasov/tmp/phones_many/'")
print("")
print("")
print("(4) CREATE TABLE "+SB+"_"+AB+"(")
print('\n,'.join(['`'+C[i][0]+'`  ' + (C[i][1] if len(C[i][1]) else 'string') for i in range(len(C))]))
print(",")
print('\n,'.join(['`'+a[i][0]+'`  ' + (a[i][1] if len(a[i][1]) else 'string') for i in range(len(a))]))
print(") STORED AS ORC tblproperties (\"orc.compress\"=\"SNAPPY\")")
print("")
print("")
print("(5) INSERT INTO "+SB+"_"+AB)
print("SELECT")
print('\n,'.join(['case when `'+C[i][0]+'`=\'\' then null else '+(C[i][2] if len(C[i][2]) else C[i][0])+' end as '+C[i][0] for i in range(len(C))]))
print(',"" as load_tags')
print(',row_number() over() as load_rank')
print(" FROM "+SB+"_"+TB)
print("")
print("")
print("(6) rename "+WARE+"/igDOL_"+SB+"."+BC)
print("part.csv to")
print("dict.csv")
print("")
print("")
print(Z)